<a href="https://colab.research.google.com/github/GenAIHub/agents-workshop/blob/main/01_basic_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Start

In this quick start, we will start with a basic chatbot that can answer common questions using an LLM, introducing key LangGraph concepts along the way. 

## Setup
Install the required packages:

In [ ]:
%%capture --no-stderr
%pip install -U langchain langchain-core langchain-openai langchain-community 
%pip install -U langgraph

Set API keys

In [ ]:
import os

# Set environment variables
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://app-ads-sbx-openai-sw.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"


# Build a Basic Chatbot

We'll first create a simple chatbot using LangGraph, that will respond directly to user messages. Though simple, it will illustrate the core concepts of building with LangGraph. 

## Start by creating a `StateGraph`. 
A `StateGraph` object defines the structure of our chatbot as a graph with a shared state.

We'll add:
- `nodes` to represent our agents
- `edges` to specify how the bot should transition between these agents.

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages represents the chat history of our agent
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

# Define our graph with the given State
graph_builder = StateGraph(State)

<div class="admonition tip">
    <p class="admonition-title">Note:</p>
    <p>
    The first thing you do when you define a graph is define the <code>State</code> of the graph. 
    The <code>State</code> consists of the schema of the graph as well as reducer functions which specify how to apply updates to the state. In our example <code>State</code> is a <code>TypedDict</code> with a single key: <code>messages</code>. The <code>messages</code> key is annotated with the <a href="https://langchain-ai.github.io/langgraph/reference/graphs/?h=add+messages#add_messages"><code>add_messages</code></a> reducer function, which tells LangGraph to append new messages to the existing list, rather than overwriting it. State keys without an annotation will be overwritten by each update, storing the most recent value.
    </p>
</div>

So now our graph knows two things:

1. Every `node` we define will receive the current `State` as input and return a value that updates that state.
2. `messages` will be _appended_ to the current list, rather than directly overwritten. This is communicated via the prebuilt [`add_messages`](https://langchain-ai.github.io/langgraph/reference/graphs/?h=add+messages#add_messages) function in the `Annotated` syntax.


## Adding the Chatbot Node
Nodes represent units of work. They are typically regular python functions.

In [ ]:
from langchain_openai import AzureChatOpenAI

# Fetching environment variables
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

if not all([api_key, endpoint, api_version, deployment_name]):
    raise ValueError("One or more environment variables are missing.")

# Initialize the Azure LLM
llm = AzureChatOpenAI(
    openai_api_key=api_key,
    azure_endpoint=endpoint,
    azure_deployment=deployment_name,
    openai_api_version=api_version,
)

def chatbot(state: State):
    # Invoke the LLM with the current chat history to get a response
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)


**Notice** how the `chatbot` node function takes the current `State` as input and returns a dictionary containing an updated `messages` list under the key "messages". This is the basic pattern for all LangGraph node functions.

The `add_messages` function in our `State` will append the llm's response messages to whatever messages are already in the state.


## Setting the Entry and Finish Points
Next, add an `entry` point. This tells our graph **where to start its work** each time we run it.

In [ ]:
graph_builder.add_edge(START, "chatbot")

Similarly, set a `finish` point. This instructs the graph **"any time this node is run, you can exit."**

In [ ]:
graph_builder.add_edge("chatbot", END)

## Compile the Graph
Finally, we'll want to be able to run our graph. To do so, call "`compile()`" on the graph builder. This creates a "`CompiledGraph`" we can use invoke on our state.

In [ ]:
graph = graph_builder.compile()

## Visualizing the Graph
You can visualize the graph using the `get_graph` method and one of the "draw" methods, like `draw_ascii` or `draw_png`. The `draw` methods each require additional dependencies.

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## Now let's run the chatbot! 

**Tip:** You can exit the chat loop at any time by typing "quit", "exit", or "q". <br>
**Note:** This implementation does not keep track of chat history though different calls of the graph. As a result, the system does not remember previous interactions with the user.

In [ ]:
while True:
    # Take user input
    user_input = input("\n\nUser: ")
    # Check whether or not to exit the loop
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    # Call the graph with a 'State' object containing the user input
    # The chat history of the previous calls to the graph is not passed!
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

## Exercise (optional):

Extend the code above such that the chat history of previous graph calls is passed to the next graph call.
As an end-user, you should be able to ask questions regarding previous interactions if implemented correctly.
You may test your solution by stating something in the first interaction 
and asking the system to repeat what you said in the second interaction. <br>
<br> **Hint:** *value["messages"]* does not return the entire chat history of a call to the graph. Instead it only returns the responses of the calls to the LLM. So you should ensure that you add the user_input to the chat history first.

## Solution:

Your answer may differ as multiple solutions exist.

In [ ]:
# Keep track of the histroy so far
# Start with an empty history
history = {"messages": []}

while True:
    # Take user input
    user_input = input("\n\nUser: ")
    # Add the user input to the history
    history['messages'] += [("user", user_input)] 
    # Check whether or not to exit the loop
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    # Take the chat histroy by accessing the messages in its state
    history_messages = history["messages"] 
    # Pass the entire conversation history to the graph
    for event in graph.stream({"messages": history_messages}): 
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)
            # Add the messages present in the event to the chat history
            # These messages only contain the response of the LLM
            history['messages'] += value['messages'] 
            

## Note:

Keeping track of the history with your multi-agent system manually is a hassle. Luckily, LangGraph provides an abstraction such that it manages and keeps track of its memory across different invocations by itself!

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

# The config allows our system to run interactions with different users 
# and still keep track of the memory for each user separately!
config = {"configurable": {"thread_id": "1"}}

In [ ]:
while True:
    # Take user input
    user_input = input("\n\nUser: ")
    # Check whether or not to exit the loop
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    # The chat history of the previous calls to the graph is not passed!
    # But due to the memory saver, the system will now manage its own memory!
    # we run each graph call with the same config, as all interactions are performed by the same end-user.
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

## **Congratulations!** 
You've built your first chatbot using LangGraph. This bot can engage in basic conversation by taking user input and generating responses using an LLM. 

However, you may have noticed that the bot's knowledge is limited to what's in its training data. In the next part, we'll add a web search tool to expand the bot's knowledge and make it more capable.